In [1]:
import pandas as panda
from pyecharts import *
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK
CurrentConfig.ONLINE_HOST
from pyecharts.globals import ThemeType
#如果是pyecharts更早的版本可能不需要运行下面的命令
import pyecharts
pyecharts.globals._WarningControl.ShowWarning = False

import warnings
warnings.filterwarnings('ignore')
from pyecharts import options as opts
#隐藏警告
from pyecharts.charts import Geo
from collections import Counter
from pyecharts.globals import GeoType
from pyecharts.globals import ChartType, SymbolType
from pyecharts.charts import BMap

In [2]:
data_2015 = pd.read_csv('Chicago_data_2015(264709, 13).csv',encoding = 'gbk')
data_2016 = pd.read_csv('Chicago_data_2016(269760, 13).csv',encoding = 'gbk')
data_2017 = pd.read_csv('Chicago_data_2017(269007, 13).csv',encoding = 'gbk')
data_2018 = pd.read_csv('Chicago_data_2018(268689, 13).csv',encoding = 'gbk')
data_2019 = pd.read_csv('Chicago_data_2019(261150, 13).csv',encoding = 'gbk')
data_2020 = pd.read_csv('Chicago_data_2020(211937, 13).csv',encoding = 'gbk')
data_2021 = pd.read_csv('Chicago_data_2021(207958, 13).csv',encoding = 'gbk')
data = pd.concat([data_2015,data_2016,data_2017,data_2018,data_2019,data_2020,data_2021])
data.shape

(1753210, 14)

In [3]:
data = data[data['案件类型']=='致幻毒品']
data.shape

(90094, 14)

In [4]:
data = data.dropna(subset=['经度','纬度'])
data.shape

(85490, 14)

In [5]:
data_select = data[['案件编号','经度','纬度','案件发生时间','案件类型']]
data_select['月份'] = pd.DatetimeIndex(data_select['案件发生时间']).month
data_select['年份'] = pd.DatetimeIndex(data_select['案件发生时间']).year
data_select['年份月份'] = [str(i)+'-'+str(j) for i,j in zip(data_select.年份,data_select.月份)]
data_select['星期'] = pd.DatetimeIndex(data_select['案件发生时间']).weekday
data_select['日'] = pd.DatetimeIndex(data_select['案件发生时间']).day
data_select['小时'] = pd.DatetimeIndex(data_select['案件发生时间']).hour
data_select['分钟'] = pd.DatetimeIndex(data_select['案件发生时间']).minute
data_select.head()

,案件编号,经度,纬度,案件发生时间,案件类型,月份,年份,年份月份,星期,日,小时,分钟
2,HY411595,-87.716650,41.937406,09/05/2015 12:45:00 PM,致幻毒品,9,2015,2015-9,5,5,12,45
37,HY411661,-87.628299,41.770447,09/05/2015 02:35:00 PM,致幻毒品,9,2015,2015-9,5,5,14,35
45,HY411709,-87.701850,41.872819,09/05/2015 03:20:00 PM,致幻毒品,9,2015,2015-9,5,5,15,20
57,HY411732,-87.633298,41.671990,09/05/2015 03:40:00 PM,致幻毒品,9,2015,2015-9,5,5,15,40
68,HY411777,-87.648699,41.790159,09/05/2015 04:16:00 PM,致幻毒品,9,2015,2015-9,5,5,16,16


In [6]:
# data.columns=['月份','jd1','wd1','案件发生地址','现场事故发生的名称','犯罪类型']
data = data_select[['案件编号','年份','案件发生时间','经度','纬度']]
# data.columns=['案件编号','案件类型','案件发生时间','经度','纬度']
data.info()
data.head()
data = data.dropna(subset=['经度','纬度'])
data = data.drop_duplicates(subset=['案件编号'], keep='first', )
data.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85490 entries, 2 to 207529
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   案件编号    85490 non-null  object 
 1   年份      85490 non-null  int64  
 2   案件发生时间  85490 non-null  object 
 3   经度      85490 non-null  float64
 4   纬度      85490 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 3.9+ MB


,案件编号,年份,案件发生时间,经度,纬度
2,HY411595,2015,09/05/2015 12:45:00 PM,-87.716650,41.937406
37,HY411661,2015,09/05/2015 02:35:00 PM,-87.628299,41.770447
45,HY411709,2015,09/05/2015 03:20:00 PM,-87.701850,41.872819
57,HY411732,2015,09/05/2015 03:40:00 PM,-87.633298,41.671990
68,HY411777,2015,09/05/2015 04:16:00 PM,-87.648699,41.790159


In [10]:
def criminal_distribution(i):
    data1=data[data['年份']==i]
#     data1 = data 
#     print('可选择的犯罪类型包括',data['犯罪类型'].unique())
    g = Geo()
    BAIDU_AK = "WrFzGI7rvGi7ylhI7oKspSmsvNYzGSQW"
    b1=BMap(init_opts=opts.InitOpts(width='1430px',height='810px'))
    b1.add_schema(
                baidu_ak=BAIDU_AK,
                center=[data1['经度'].mean(),data1['纬度'].mean()],
                zoom=12
                )
    g.add_schema()
    for row in data1.index:
        g.add_coordinate(str(data1.loc[row]['案件编号']),data1.loc[row]['经度'],data1.loc[row]['纬度'])
    data_pair = []
    for row in data1.index:
        data_pair.append((str(data1.loc[row]['案件编号']),1))
    b1.add('',
           data_pair,
           label_opts=opts.LabelOpts(is_show = False),
           type_=GeoType.SCATTER,
           symbol_size=3)
    b1.set_global_opts(
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True,max_=1),   #注意默认max_
        title_opts=opts.TitleOpts(title="{}年{}毒品类犯罪分布情况".format(str(i),'芝加哥市'),pos_left='left',title_textstyle_opts=opts.TextStyleOpts(font_size=35,font_weight='bold')),
    )
    b1.render('芝加哥市毒品类犯罪分布.html')

In [11]:
print('可选的年份:',data['年份'].unique())

可选的年份: [2015 2016 2017 2018 2019 2020 2021]


In [18]:
i = input()
criminal_distribution(int(i))
print('已在当前目录下生成对应的HTML文件，名为“芝加哥市毒品类犯罪分布.html”，快去点开查看一下吧，欢迎使用本函数！再次使用时仅需要刷新该文件即可，不会占用更多内存！')

2021
已在当前目录下生成对应的HTML文件，名为“芝加哥市毒品类犯罪分布.html”，快去点开查看一下吧，欢迎使用本函数！再次使用时仅需要刷新该文件即可，不会占用更多内存！
